### Load Context

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

os.chdir("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/")
sys.path.append("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/src")

from kedro.framework.session import KedroSession

with KedroSession.create("biblical_scripts") as session:
    ctx = session.load_context()

params = ctx.params


/usr/local/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.8/site-packages/statsmodels/iolib/foreign.py:651: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/pyth

2021-11-11 12:35:38,838 - kedro.framework.session.store - INFO - `save()` not implemented for `BaseSessionStore`. Skipping the step.


# chunk_len --  Text Length vs Accuracy

In [17]:
sim_res = ctx.catalog.load('sim_len_res')

2021-11-11 12:41:28,798 - kedro.io.data_catalog - INFO - Loading data from `sim_len_res` (CSVDataSet)...


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [9]:
def _eval_succ(df) :
    """
    Indicate whetehr minimal discripancy is obtained by the true author.
    """
    idx_min = df.groupby(['doc_id', 'author'])['value'].idxmin()
    res_min = df.loc[idx_min, :].rename(columns={'corpus' : 'most_sim'})
    res_min.loc[:, 'succ'] = res_min.author == res_min.most_sim
    return res_min


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [26]:
value = "HC"
df1 = sim_res[sim_res['variable'].str.contains(f":{value}")]
df1.loc[:,'corpus'] = df1['variable'].str.extract(r'([^:]+):')[0]
df1['author'] = df1['true_author']
df1['doc_id'] = df1['experiment'] + ":" + df1['true_author'] \
            + ":" + df1['itr'].astype(str) + ":" + df1['chunk_size'].astype(str)
df1 = df1.reset_index()

_eval_succ(df1)

grp = df_res.groupby('chunk_size')
for c in grp :
   a = c[1]['succ'].agg(['mean', 'std',
                           lambda x : pd.Series.quantile(x, q=.05),
                           lambda x : pd.Series.quantile(x, q=.95)
                          ])

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [43]:
df_res.groupby(['chunk_size']).succ.agg(['mean'], as_index=False)


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,mean
chunk_size,
5,0.660000
10,0.750000
15,0.800000
20,0.826667
25,0.830000
30,0.803333
40,0.836667
50,0.790000
60,0.866667


In [5]:
value = "HC"
df1 = sim_res[sim_res['variable'].str.contains(f":{value}")]
df1.loc[:,'corpus'] = df1['variable'].str.extract(r'([^:]+):')[0]
df1['author'] = df1['true_author']
df1['doc_id'] = df1['experiment'] + ":" + df1['true_author'] \
            + ":" + df1['itr'].astype(str) + ":" + df1['chunk_size'].astype(str)
df1 = df1.reset_index()

df_res = _eval_succ(df1)

# average over chunk_len
df_res['succ'] = df_res['succ'] + .0
grp = df_res.groupby('chunk_size')
res = grp.agg({'succ' : ['mean', 'std',
                           lambda x : pd.Series.quantile(x, q=.05),
                           lambda x : pd.Series.quantile(x, q=.95)
                          ]}, as_index=False).reset_index()

res[f'succ_mean'] = res[('succ', 'mean')]
res[f'succ_std'] = res[('succ', 'std')]
res[f'succ_CI_lower'] = res[('succ', '<lambda_0>')]
res[f'succ_CI_upper'] = res[('succ', '<lambda_1>')]
res = res.drop('succ', axis=1, level=0)
res

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,chunk_size,succ_mean,succ_std,succ_CI_lower,succ_CI_upper
,,,,,
0,5,0.660000,0.474500,0.0,1.0
1,10,0.750000,0.433736,0.0,1.0
2,15,0.800000,0.400668,0.0,1.0
3,20,0.826667,0.379168,0.0,1.0
4,25,0.830000,0.376260,0.0,1.0
5,30,0.803333,0.398142,0.0,1.0
6,40,0.836667,0.370287,0.0,1.0
7,50,0.790000,0.407989,0.0,1.0
8,60,0.866667,0.340503,0.0,1.0


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,author,doc_id,len,variable,value,doc_tested,true_author,chunk_size,itr,experiment,corpus
0,NaN,<TEST>,135.0,HC,-6.057500,<TEST>,Dtr,5,0,chunk_len,Dtr
1,NaN,<TEST>,135.0,HC,-2.309598,<TEST>,Dtr,5,0,chunk_len,DtrH
2,NaN,<TEST>,135.0,HC,1.056556,<TEST>,Dtr,5,0,chunk_len,P
3,NaN,<TEST>,122.0,HC,0.800869,<TEST>,DtrH,5,0,chunk_len,Dtr
4,NaN,<TEST>,122.0,HC,0.348251,<TEST>,DtrH,5,0,chunk_len,DtrH
...,...,...,...,...,...,...,...,...,...,...,...
11695,NaN,<TEST>,2577.0,HC,2.146651,<TEST>,DtrH,100,99,chunk_len,DtrH
11696,NaN,<TEST>,2577.0,HC,7.144911,<TEST>,DtrH,100,99,chunk_len,P
11697,NaN,<TEST>,1887.0,HC,8.288324,<TEST>,P,100,99,chunk_len,Dtr
11698,NaN,<TEST>,1887.0,HC,8.198978,<TEST>,P,100,99,chunk_len,DtrH


In [8]:
sim_res

,author,doc_id,len,variable,value,doc_tested,true_author,chunk_size,itr,experiment
0,<TEST>,<TEST>,135.0,Dtr:HC,-6.057500,<TEST>,Dtr,5,0,chunk_len
1,<TEST>,<TEST>,135.0,DtrH:HC,-2.309598,<TEST>,Dtr,5,0,chunk_len
2,<TEST>,<TEST>,135.0,P:HC,1.056556,<TEST>,Dtr,5,0,chunk_len
3,<TEST>,<TEST>,122.0,Dtr:HC,0.800869,<TEST>,DtrH,5,0,chunk_len
4,<TEST>,<TEST>,122.0,DtrH:HC,0.348251,<TEST>,DtrH,5,0,chunk_len
...,...,...,...,...,...,...,...,...,...,...
11695,<TEST>,<TEST>,2577.0,DtrH:HC,2.146651,<TEST>,DtrH,100,99,chunk_len
11696,<TEST>,<TEST>,2577.0,P:HC,7.144911,<TEST>,DtrH,100,99,chunk_len
11697,<TEST>,<TEST>,1887.0,Dtr:HC,8.288324,<TEST>,P,100,99,chunk_len
11698,<TEST>,<TEST>,1887.0,DtrH:HC,8.198978,<TEST>,P,100,99,chunk_len
